<a href="https://colab.research.google.com/github/Matvezy/Body-Mesurments/blob/master/testingSUCCESS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [60]:
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [61]:
%cd /content/gdrive/MyDrive/AI_500_512px_SizesOnly_Aligned

/content/gdrive/MyDrive/AI_500_512px_SizesOnly_Aligned


In [62]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Wed Jan 13 03:46:17 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.27.04    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    31W / 250W |   1755MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [63]:
def findppm(mask, height):
  mask = mask.cpu().numpy()[0]
  masklen = len(mask) - 1
  trig1 = False
  trig2 = False
  while masklen >= 0:
    for j in mask[masklen]:
      if j > 0.5:
        trig1 = True
        break
    if trig1:
      break
    masklen -= 1
  topix = 0
  while topix < len(mask):
    for j in mask[topix]:
      if j > 0.5:
        trig2 = True
        break
    if trig2:
      break 
    topix += 1
  pixheight = masklen - topix
  ppmratio = pixheight/height
  return ppmratio

In [64]:
import torch
import torch.nn as nn
from torchvision import datasets, models, transforms
class Network(nn.Module):
    def __init__(self,num_classes=12):
        super().__init__()
        self.model_name='resnet34'
        self.model=models.resnet34()
        self.model.conv1=nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.model.fc=nn.Linear(self.model.fc.in_features, num_classes)
        
    def forward(self, x):
        x=self.model(x)
        return x

In [65]:
def gettest(frontPPM, h_frontDot1, h_frontDot2, w_frontDot1, w_frontDot2, b_frontDot1, b_frontDot2, sidePPM, h_sideDot1, h_sideDot2, w_sideDot1, w_sideDot2, b_sideDot1, b_sideDot2):
  """
  model1 = torch.load('/content/gdrive/MyDrive/AI_1050_512px_SizesRelative_Aligned/hipspred_model.pt')
  model1.cuda()
  model1.eval()
  res =  model1(torch.tensor([frontPPM,	frontDot1,	frontDot2,	sidePPM,	sideDot1,	sideDot2]).cuda())
  print("Predicted Hips Loop: "+str(res.item()))
  """
  hips = (((h_frontDot1-h_frontDot2)/frontPPM)*2 + ((h_sideDot1-h_sideDot2)/sidePPM)*1.3)
  waist = (((w_frontDot1-w_frontDot2)/frontPPM)*2 + ((w_sideDot1-w_sideDot2)/sidePPM)*1.3)
  bust = (((b_frontDot1-b_frontDot2)/frontPPM)*2 + ((b_sideDot1-b_sideDot2)/sidePPM)*1.3)
  return hips, waist, bust

In [66]:
import cv2
from PIL import Image
import torchvision.transforms.functional as TF
import torchvision.transforms as T
from torch.autograd import Variable
def image_loader(image_name):
    image = cv2.imread(image_name, 0)
    image = Image.fromarray(image)
    image = TF.resize(image, (512,512))
    image = TF.to_tensor(image)
    image = Variable(image, requires_grad=True)
    return image.cuda()  #assumes that you're using GPU

In [67]:
def run_img(image1, image2, height):
  with torch.no_grad():
    img = image_loader(image1)
    front_network = Network()
    front_network.cuda()
    front_network.load_state_dict(torch.load('/content/gdrive/MyDrive/AI_1050_512px_SizesRelative_Aligned/front_landmarks5.pth')) 
    front_network.eval()
    predictions = (front_network(img[None, ...]).cpu())
    predictions = predictions.view(-1,6,2)
    predictions = predictions.detach().numpy()
    h_frontDot1 = predictions[0][2][0]
    h_frontDot2 = predictions[0][5][0]
    w_frontDot1 = predictions[0][1][0]
    w_frontDot2 = predictions[0][4][0]
    b_frontDot1 = predictions[0][0][0]
    b_frontDot2 = predictions[0][3][0]
    img1 = image_loader(image2)
    side_network = Network()
    side_network.cuda()
    side_network.load_state_dict(torch.load('/content/gdrive/MyDrive/AI_1050_512px_SizesRelative_Aligned/side_landmarks6.pth')) 
    side_network.eval()
    predictions1 = (side_network(img1[None, ...]).cpu())
    predictions1 = predictions1.view(-1,6,2)
    predictions1 = predictions1.detach().numpy()
    h_sideDot1 = predictions1[0][5][0]
    h_sideDot2 = predictions1[0][0][0]
    w_sideDot1 = predictions1[0][4][0]
    w_sideDot2 = predictions1[0][1][0]
    b_sideDot1 = predictions1[0][3][0]
    b_sideDot2 = predictions1[0][2][0]
    fimg = image1
    fimg  = Image.open(fimg)
    transform = T.Compose([T.ToTensor()])
    fimg = transform(fimg)
    front_ppm = findppm(fimg, height)

    simg = image2
    simg  = Image.open(simg)
    simg = transform(simg)
    side_ppm = findppm(simg, height)

    hips, waist, bust = gettest(front_ppm, h_frontDot1, h_frontDot2, w_frontDot1, w_frontDot2, b_frontDot1, b_frontDot2, side_ppm, h_sideDot1, h_sideDot2, w_sideDot1, w_sideDot2, b_sideDot1, b_sideDot2)
    return hips, waist, bust

In [68]:
hips_res = []
waist_res = []
bust_res = []
for i in range(1, 500):
  metrics = open('AITrainingLib/'+str(i)+'/Data.txt','r').read().split("\n")
  height = int(metrics[0].split(": ")[1])
  hips_loop = int(metrics[1].split(": ")[1])
  waist_loop = int(metrics[3].split(": ")[1])
  bust_loop = int(metrics[5].split(": ")[1])
  pred_hips, pred_waist, pred_bust = run_img('AITrainingLib/'+str(i)+'/Front.png', 'AITrainingLib/'+str(i)+'/Side.png', height)
  hips_res.append(abs(hips_loop - pred_hips))
  waist_res.append(abs(waist_loop - pred_waist))
  bust_res.append(abs(bust_loop - pred_bust))
  if i%100 == 0:
    print('Height:'+str(height))
    print('Actual Hips Loop:'+str(hips_loop))
    print("Predicted Hips Loop: "+str(pred_hips))
    print('Actual Waist Loop:'+str(waist_loop))
    print("Predicted Waist Loop: "+str(pred_waist))
    print('Actual Bust Loop:'+str(bust_loop))
    print("Predicted Bust Loop: "+str(pred_bust))

Height:180
Actual Hips Loop:99
Predicted Hips Loop: 100.39055822687776
Actual Waist Loop:80
Predicted Waist Loop: 77.50315733920323
Actual Bust Loop:79
Predicted Bust Loop: 74.5903325284465
Height:180
Actual Hips Loop:103
Predicted Hips Loop: 104.60072703711313
Actual Waist Loop:60
Predicted Waist Loop: 71.16830568792277
Actual Bust Loop:82
Predicted Bust Loop: 75.96142595062844
Height:180
Actual Hips Loop:93
Predicted Hips Loop: 94.12778581707218
Actual Waist Loop:66
Predicted Waist Loop: 74.44822056944699
Actual Bust Loop:100
Predicted Bust Loop: 95.31775262900915
Height:180
Actual Hips Loop:114
Predicted Hips Loop: 113.69934814790363
Actual Waist Loop:90
Predicted Waist Loop: 89.42348202858729
Actual Bust Loop:107
Predicted Bust Loop: 107.15766531011371


In [69]:
print("Average Hips Loop Error: " + str(sum(hips_res) / len(hips_res)) +" (cm)")
print("Average Waist Loop Error: " + str(sum(waist_res) / len(waist_res)) +" (cm)")
print("Average Bust Loop Error: " + str(sum(bust_res) / len(bust_res)) +" (cm)")

Average Hips Loop Error: 1.2080626173616853 (cm)
Average Waist Loop Error: 5.190086949341243 (cm)
Average Bust Loop Error: 2.9545671270449505 (cm)
